# init

In [6]:
import pygame
import numpy as np
import numpy.linalg as lin
import sys
import importlib
import itertools
import time

import Clipping

import Geometry
from Geometry import Point
from camera import Camera
import draw
from inputs import input_update

# reload modules

In [136]:

importlib.reload(Clipping)
importlib.reload(Geometry)
importlib.reload(sys.modules['vec'])
importlib.reload(sys.modules['util'])
importlib.reload(sys.modules['opengl_draw_2d'])
importlib.reload(sys.modules['opengl_draw_3d'])
import Geometry

importlib.reload(sys.modules['camera'])
from camera import Camera
importlib.reload(sys.modules['inputs'])
from inputs import input_update

importlib.reload(sys.modules['draw'])
import draw



# build shapes

In [122]:
from colors import *
    
d = 3 #set dimensionality
cube_face_colors = [RED,YELLOW,GREEN,CYAN,MAGENTA, WHITE]
tess_face_colors = cube_face_colors + [ORANGE,PURPLE]

cube_shape = Geometry.build_cube(d)

#draw a pair of cubes for each axis
for face, color in zip(cube_shape.faces,tess_face_colors):
    face.color = color
shapes = []
r = 2
for i in range(d):
    cube = cube_shape.copy()
    cube.update(pos = r*np.eye(d)[i])
    shapes.append(cube)
    cube = cube_shape.copy()
    cube.update(pos = -r*np.eye(d)[i])
    shapes.append(cube)
shapes = [shapes[-2]] 
#shapes.pop(2) #removes top cube
#cube_shape.transparent = True
    
for i,shape in enumerate(shapes):
    shape.name = 'shape ' + str(i)

shape_combinations = list(itertools.combinations(shapes,2))

# main loop

Would like draw.py to have clearer distinctions between the various spaces. The layers are
1. The d-dim local spaces that shape.verts_ref, ref_frame, and face.normal_ref etc live in
2. The d-dim world space that rotated, scaled and positioned shapes (and faces) and cameras live
3. The d-dim space rotated and translated wrt to a camera
4. The (d-1)-dim space that shapes are projected on to
5. The (d-1)-dim graphics backend (openGL)

It would be nice if we could organize code according to these layers.

Clipping happens in 2

In [24]:
import colors
import vec
def test_points(camera,pos):
    points = [ vec.Vec([np.cos(th),np.sin(th),0,0]) + pos for th in np.linspace(0,2*np.pi,200)]
    draw.draw_points(camera,points,colors.GREEN)

In [137]:
#does not draw before input / does not update camera properly?

t = 0
#size = width, height = 1100, 700
size = width, height = 1600, 900

pygame.init()

if d == 3:
    draw.init(d,size,focal=6,n_fuzz_points = 200)
    draw.view_radius = 7
    draw.view_boundary = 'cylinder'
    camera = Camera(pos = Point([3,3,3]))
    #camera.update_rot_matrix(0,-1,-3*np.pi/4)
    #camera.update_rot_matrix(1,-1,-np.pi/4)
    #camera.look_at(vec.zero_vec(d))
    camera.look_at(shapes[0].pos)
    #camera = Camera(pos = Point([0,0,0]))
if d == 4:
    draw.init(d,size,stereo=False,n_fuzz_points = 200)
    draw.view_boundary = 'cylinder'
    if draw.stereo:
        draw.view_radius = 4
    else:
        draw.view_radius = 6
    #camera = Camera(pos = Point([0,0,0,0]))
    #draw.face_scales = [0.95,0.5]
    #draw_class.stereo_view_angles = [[-30,0],[0,0]]
    #camera = Camera(pos = Point([0,0,0,0]))
    camera = Camera(pos = vec.ones_vec(d))
    camera.look_at(vec.zero_vec(d))
    #camera.update_rot_matrix(0,-1,-3*np.pi/4)
    #camera.update_rot_matrix(2,-1,-np.pi/4)
    #camera.update_rot_matrix(1,-1,-np.pi/4)
#camera.update_rot_matrix(2,3,-np.pi/4)
draw.clipping = True
quit = False
redraw = True
timer = time.time()
frame = 0
try:
    while not quit:

        if redraw:
            draw.draw(camera,shapes)
            #test_points(camera,shapes[0].pos)
            pygame.display.flip()
            redraw = False
            
        #this is here so that we draw on the first frame
        redraw, quit = input_update(camera, shapes)
        #shapes[0].update(angles=[0,time.time()]) #rotate shapes[0]
        redraw = True #redraw every loop. useful for assessing framerate


        delta_t = (time.time() - timer)
        if delta_t > 2:
        #if False:
            #print('redraw:',redraw)
            try:
                #fps = 30/(time.time() - timer)
                fps = frame/delta_t
            except:
                fps = 0
            #if fps < 60:
            print(fps,'fps')
            timer = time.time()
            frame = 0
        frame = frame+1
except:
    pygame.quit()
    print('game terminated')
    raise
pygame.quit()
print('game terminated')

init GL
225.32926930955273 fps
238.3705935855879 fps
221.0378251727188 fps
225.93630178526377 fps
229.93036879012627 fps
221.49461354003404 fps
220.25909266767192 fps
211.71732687223326 fps
217.44784533204935 fps
217.16418102309328 fps
214.68475260941472 fps
220.90874063435524 fps
224.3073150500952 fps
231.16944724087068 fps
226.67652027318593 fps
224.40358942314612 fps
219.96109766017057 fps
216.4086237734073 fps
218.46634841616208 fps
game terminated


In [24]:
pygame.quit()

In [118]:
vec.insert_index(vec.drop_index([1,2,3],1),1,2)

array([1, 2, 3])

In [79]:
shapes[0].verts_ref

[array([-1., -1., -1., -1.]),
 array([-1., -1., -1.,  1.]),
 array([-1., -1.,  1., -1.]),
 array([-1., -1.,  1.,  1.]),
 array([-1.,  1., -1., -1.]),
 array([-1.,  1., -1.,  1.]),
 array([-1.,  1.,  1., -1.]),
 array([-1.,  1.,  1.,  1.]),
 array([ 1., -1., -1., -1.]),
 array([ 1., -1., -1.,  1.]),
 array([ 1., -1.,  1., -1.]),
 array([ 1., -1.,  1.,  1.]),
 array([ 1.,  1., -1., -1.]),
 array([ 1.,  1., -1.,  1.]),
 array([ 1.,  1.,  1., -1.]),
 array([1., 1., 1., 1.])]